### 原代码

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import seaborn as sns
#包的准备

: 

In [ ]:
class stock_info:
    def __init__(self, filepath, target_col):
        # Load data from HDF file
        self.raw = pd.read_hdf(filepath)
        
        # Extract the target column i.e "IC"
        self.target_raw = self.raw[target_col]
        
        # Get the list of dates
        self.dates = self.target_raw.index.tolist()
        
        # initiate a new dataframe recording daily return 
        self.readouts = pd.DataFrame(self.dates, columns=["dates"])
        self.readouts.set_index("dates", inplace=True)
        
        # Calculate T_last_close and T_vap (VWAP)
        self.readouts["T_last_close"] = [
            self.target_raw.loc[date]["Close"].iloc[-1] for date in self.dates
        ]
        self.readouts["T_vap"] = [
            (self.target_raw.loc[date]["Close"] * self.target_raw.loc[date]["Volume"]).sum() /
            self.target_raw.loc[date]["Volume"].sum()
            for date in self.dates
        ]
        
        # Calculate default strategy
        # if the last close price is higher than VWAP (T_vap), sell (-1), otherwise buy (+1)
        ratio = np.array(self.readouts["T_last_close"]) / np.array(self.readouts["T_vap"])
        self.readouts["default_st"] = np.where(ratio > 1., -1, 1)

    def evaluate_return_T_p1(self, t_exec, strategy_array_name="default_st"):
        # read strategy for T+1
        strategy_ = np.array(self.readouts[strategy_array_name].tolist()[:-2])
        
        # read close price at specified time in T+1 and T+2 respectively
        short_price_T_p1 = np.array([self.target_raw.loc[day]["Close"].iloc[t_exec] for day in self.dates[1:-1]])
        long_price_T_p2 = np.array([self.target_raw.loc[day]["Close"].iloc[t_exec] for day in self.dates[2:]])
        
        # calculate return rate at T+1
        return_rate_T_p1 = pd.Series(((long_price_T_p2/short_price_T_p1)-1.)*strategy_, index=self.dates[:-2])
        
        # append to self.readouts
        self.readouts["return_"+strategy_array_name+"_"+str(t_exec)] = return_rate_T_p1

    def get_sharpe_ratio(self, strategy_return):
        # obtain return rate under specified strategy
        return_arr = self.readouts[strategy_return].tolist()[:-2]
        
        # calculate traditional sharpe ratio
        sharpe_ratio = np.sqrt(250.)*np.mean(return_arr)/np.std(return_arr)
        return sharpe_ratio

    def generate_time_dependency(self, strategy_array_name):
        fig, ax1 = plt.subplots(1, 1)
        
        # array of trading time
        fibonnacci = [0, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 239]
        return_arr = []
        
        for t_exec in fibonnacci:
            # calculate return rate of combinations of strategy with different trading times
            self.evaluate_return_T_p1(t_exec, strategy_array_name=strategy_array_name)
            # record sharpe ratio
            return_arr.append(self.get_sharpe_ratio(strategy_return="return_"+strategy_array_name+"_"+str(t_exec)))
            
        # plot sensitivity    
        ax1.plot(fibonnacci, return_arr, label="sharpe ratio", c="blue")
        ax1.scatter(fibonnacci, return_arr, c="blue")
        title = "time sensitivity of " + strategy_array_name
        ax1.set_title(title)
        ax1.set_xlabel("trading time (min)")
        ax1.set_ylabel("sharpe ratio")
        plt.legend()
        plt.show()


In [ ]:
mydata = stock_info("MinutesIdx.h5", "IC")

mydata.evaluate_return_T_p1(59)
print(mydata.readouts)
mydata.generate_time_dependency("default_st")

### 固定阈值

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import seaborn as sns

# 定义类
class stock_info:
    def __init__(self, filepath, target_col):
        # 读取HDF文件
        self.raw = pd.read_hdf(filepath)
        # 提取某一个目标列(如“IC”)
        self.target_raw = self.raw[target_col]
        # 拿到所有的交易日列表
        self.dates = self.target_raw.index.tolist()
        # 初始化一个DataFrame,用于记录结果
        self.readouts = pd.DataFrame(self.dates, columns=["dates"])
        self.readouts.set_index("dates", inplace=True)

        # 计算T日收盘价和VWAP(成交量加权平均价)
        self.readouts["T_last_close"] = [
            self.target_raw.loc[date]["Close"].iloc[-1] for date in self.dates
        ]
        self.readouts["T_vap"] = [
            (self.target_raw.loc[date]["Close"] * self.target_raw.loc[date]["Volume"]).sum()
            / self.target_raw.loc[date]["Volume"].sum()
            for date in self.dates
        ]

        # ========= 策略1：原始反转策略 =========
        ratio = np.array(self.readouts["T_last_close"]) / np.array(self.readouts["T_vap"])
        self.readouts["default_st"] = np.where(ratio > 1., -1, 1)

        # ========= 策略2：固定阈值 =========
        deviation = (self.readouts["T_last_close"] - self.readouts["T_vap"]) / self.readouts["T_vap"]
        threshold = 0.001  # 固定阈值 0.1%
        self.readouts["improved_st"] = np.where(
            deviation > threshold, -1,
            np.where(deviation < -threshold, 1, 0)
        )

    # 计算策略收益
    def evaluate_return_T_p1(self, t_exec, strategy_array_name="default_st"):
        strategy_ = np.array(self.readouts[strategy_array_name].tolist()[:-2])
        short_price_T_p1 = np.array([self.target_raw.loc[day]["Close"].iloc[t_exec] for day in self.dates[1:-1]])
        long_price_T_p2 = np.array([self.target_raw.loc[day]["Close"].iloc[t_exec] for day in self.dates[2:]])
        return_rate_T_p1 = pd.Series(((long_price_T_p2 / short_price_T_p1) - 1.) * strategy_,
                                     index=self.dates[:-2])
        self.readouts["return_" + strategy_array_name + "_" + str(t_exec)] = return_rate_T_p1

    # 计算夏普比率
    def get_sharpe_ratio(self, strategy_return):
        return_arr = self.readouts[strategy_return].tolist()[:-2]
        if np.std(return_arr) == 0:
            return 0
        sharpe_ratio = np.sqrt(250.) * np.mean(return_arr) / np.std(return_arr)
        return sharpe_ratio

    # 单策略敏感性分析
    def generate_time_dependency(self, strategy_array_name):
        fig, ax1 = plt.subplots(1, 1)
        fibonnacci = [0, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 239]
        return_arr = []
        for t_exec in fibonnacci:
            self.evaluate_return_T_p1(t_exec, strategy_array_name=strategy_array_name)
            return_arr.append(
                self.get_sharpe_ratio("return_" + strategy_array_name + "_" + str(t_exec))
            )
        ax1.plot(fibonnacci, return_arr, label=strategy_array_name)
        ax1.scatter(fibonnacci, return_arr)
        title = "time sensitivity of " + strategy_array_name
        ax1.set_title(title)
        ax1.set_xlabel("trading time (min)")
        ax1.set_ylabel("sharpe ratio")
        plt.legend()
        plt.show()

    # 多策略敏感性对比
    def generate_multi_time_dependency(self, strategy_list):
        fig, ax = plt.subplots(1, 1, figsize=(8, 6))
        fibonnacci = [0, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 239]

        for strategy_array_name in strategy_list:
            return_arr = []
            for t_exec in fibonnacci:
                self.evaluate_return_T_p1(t_exec, strategy_array_name=strategy_array_name)
                return_arr.append(
                    self.get_sharpe_ratio("return_" + strategy_array_name + "_" + str(t_exec))
                )
            ax.plot(fibonnacci, return_arr, label=strategy_array_name)
            ax.scatter(fibonnacci, return_arr)

        ax.set_title("Time Sensitivity Comparison of Strategies")
        ax.set_xlabel("trading time (min)")
        ax.set_ylabel("sharpe ratio")
        plt.legend()
        plt.show()


# ===== 使用示例 =====
mydata = stock_info("MinutesIdx.h5", "IC")

# 只展示 策略1 和 策略2 的对比曲线
mydata.generate_multi_time_dependency(
    ["default_st", "improved_st"]
)


### 固定阈值 ➕ 自适应阈值 ➕ 自适应阈值、趋势过滤

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import seaborn as sns

# 定义类
class stock_info:
    def __init__(self, filepath, target_col):
        # 读取HDF文件
        self.raw = pd.read_hdf(filepath)
        # 提取某一个目标列(如“IC”)
        self.target_raw = self.raw[target_col]
        # 拿到所有的交易日列表
        self.dates = self.target_raw.index.tolist()
        # 初始化一个DataFrame,用于记录结果
        self.readouts = pd.DataFrame(self.dates, columns=["dates"])
        self.readouts.set_index("dates", inplace=True)

        # 计算T日收盘价和VWAP(成交量加权平均价)
        self.readouts["T_last_close"] = [
            self.target_raw.loc[date]["Close"].iloc[-1] for date in self.dates
        ]
        self.readouts["T_vap"] = [
            (self.target_raw.loc[date]["Close"] * self.target_raw.loc[date]["Volume"]).sum()
            / self.target_raw.loc[date]["Volume"].sum()
            for date in self.dates
        ]

        # ========= 策略1：原始反转策略 =========
        ratio = np.array(self.readouts["T_last_close"]) / np.array(self.readouts["T_vap"])
        self.readouts["default_st"] = np.where(ratio > 1., -1, 1)

        # ========= 策略2：固定阈值 =========
        deviation = (self.readouts["T_last_close"] - self.readouts["T_vap"]) / self.readouts["T_vap"]
        threshold = 0.001  # 固定阈值 0.05%
        self.readouts["improved_st"] = np.where(
            deviation > threshold, -1,
            np.where(deviation < -threshold, 1, 0)
        )

        # ========= 策略3：自适应阈值 =========
        window = 7  # 最近7天波动
        rolling_std = deviation.rolling(window=window, min_periods=5).std().fillna(method="bfill")
        self.readouts["adaptive_st"] = np.where(
            deviation > rolling_std, -1,
            np.where(deviation < -rolling_std, 1, 0)
        )

        # ========= 策略4：自适应阈值 + 趋势过滤 =========
        window_ma = 54  # 均线长度
        ma = self.readouts["T_last_close"].rolling(window=window_ma, min_periods=5).mean().fillna(method="bfill")

        adaptive = self.readouts["adaptive_st"].copy()

        def trend_filter(sig, close, ma_val):
            if sig == 0:
                return 0
            elif close > ma_val and sig == 1:   # 均线上只做多
                return 1
            elif close < ma_val and sig == -1:  # 均线下只做空
                return -1
            else:
                return 0  # 反趋势信号过滤掉

        self.readouts["adaptive_trend_st"] = [
            trend_filter(sig, close, ma_val)
            for sig, close, ma_val in zip(adaptive, self.readouts["T_last_close"], ma)
        ]

    # 计算策略收益
    def evaluate_return_T_p1(self, t_exec, strategy_array_name="default_st"):
        strategy_ = np.array(self.readouts[strategy_array_name].tolist()[:-2])
        short_price_T_p1 = np.array([self.target_raw.loc[day]["Close"].iloc[t_exec] for day in self.dates[1:-1]])
        long_price_T_p2 = np.array([self.target_raw.loc[day]["Close"].iloc[t_exec] for day in self.dates[2:]])
        return_rate_T_p1 = pd.Series(((long_price_T_p2 / short_price_T_p1) - 1.) * strategy_,
                                     index=self.dates[:-2])
        self.readouts["return_" + strategy_array_name + "_" + str(t_exec)] = return_rate_T_p1

    # 计算夏普比率
    def get_sharpe_ratio(self, strategy_return):
        return_arr = self.readouts[strategy_return].tolist()[:-2]
        if np.std(return_arr) == 0:
            return 0
        sharpe_ratio = np.sqrt(250.) * np.mean(return_arr) / np.std(return_arr)
        return sharpe_ratio

    # 单策略敏感性分析
    def generate_time_dependency(self, strategy_array_name):
        fig, ax1 = plt.subplots(1, 1)
        fibonnacci = [0, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 239]
        return_arr = []
        for t_exec in fibonnacci:
            self.evaluate_return_T_p1(t_exec, strategy_array_name=strategy_array_name)
            return_arr.append(
                self.get_sharpe_ratio("return_" + strategy_array_name + "_" + str(t_exec))
            )
        ax1.plot(fibonnacci, return_arr, label=strategy_array_name)
        ax1.scatter(fibonnacci, return_arr)
        title = "time sensitivity of " + strategy_array_name
        ax1.set_title(title)
        ax1.set_xlabel("trading time (min)")
        ax1.set_ylabel("sharpe ratio")
        plt.legend()
        plt.show()

    # 多策略敏感性对比
    def generate_multi_time_dependency(self, strategy_list):
        fig, ax = plt.subplots(1, 1, figsize=(8, 6))
        fibonnacci = [0, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 239]

        for strategy_array_name in strategy_list:
            return_arr = []
            for t_exec in fibonnacci:
                self.evaluate_return_T_p1(t_exec, strategy_array_name=strategy_array_name)
                return_arr.append(
                    self.get_sharpe_ratio("return_" + strategy_array_name + "_" + str(t_exec))
                )
            ax.plot(fibonnacci, return_arr, label=strategy_array_name)
            ax.scatter(fibonnacci, return_arr)

        ax.set_title("Time Sensitivity Comparison of Strategies")
        ax.set_xlabel("trading time (min)")
        ax.set_ylabel("sharpe ratio")
        plt.legend()
        plt.show()


# ===== 使用示例 =====
mydata = stock_info("MinutesIdx.h5", "IC")

# 展示四种方法的对比曲线
mydata.generate_multi_time_dependency(
    ["default_st", "improved_st", "adaptive_st", "adaptive_trend_st"]
)

### 调参

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import seaborn as sns

# 定义类
class stock_info:
    def __init__(self, filepath, target_col):
        # 读取HDF文件
        self.raw = pd.read_hdf(filepath)
        # 提取目标列 (如 “IC”)
        self.target_raw = self.raw[target_col]
        # 所有交易日列表
        self.dates = self.target_raw.index.tolist()
        # 初始化结果 DataFrame
        self.readouts = pd.DataFrame(self.dates, columns=["dates"])
        self.readouts.set_index("dates", inplace=True)

        # T日收盘价和VWAP
        self.readouts["T_last_close"] = [
            self.target_raw.loc[date]["Close"].iloc[-1] for date in self.dates
        ]
        self.readouts["T_vap"] = [
            (self.target_raw.loc[date]["Close"] *
             self.target_raw.loc[date]["Volume"]).sum() /
            self.target_raw.loc[date]["Volume"].sum()
            for date in self.dates
        ]

        # 策略1: 原始 (无阈值)
        ratio = np.array(self.readouts["T_last_close"]) / np.array(self.readouts["T_vap"])
        self.readouts["default_st"] = np.where(ratio > 1., -1, 1)

        # 策略2: 固定阈值 (0.2%)
        threshold = 0.002
        deviation = (self.readouts["T_last_close"] - self.readouts["T_vap"]) / self.readouts["T_vap"]
        self.readouts["improved_st"] = np.where(
            deviation > threshold, -1,
            np.where(deviation < -threshold, 1, 0)
        )

        # 策略3: 自适应阈值 (rolling std, 默认7天)
        adaptive_window = 7
        rolling_std = deviation.rolling(window=adaptive_window, min_periods=1).std().fillna(method="bfill")
        self.readouts["adaptive_st"] = np.where(
            deviation > rolling_std, -1,
            np.where(deviation < -rolling_std, 1, 0)
        )

        # 策略4: 自适应 + 趋势过滤 (默认 MA=20)
        ma_len = 20
        ma = self.readouts["T_last_close"].rolling(window=ma_len, min_periods=1).mean().fillna(method="bfill")
        close = self.readouts["T_last_close"].values
        adaptive = self.readouts["adaptive_st"].values
        trend_sig = np.zeros_like(adaptive)
        for i in range(len(adaptive)):
            sig = int(adaptive[i])
            if sig == 0:
                trend_sig[i] = 0
            else:
                if sig == 1 and close[i] > ma.iat[i]:
                    trend_sig[i] = 1
                elif sig == -1 and close[i] < ma.iat[i]:
                    trend_sig[i] = -1
                else:
                    trend_sig[i] = 0
        self.readouts["adaptive_trend_st"] = trend_sig

    # 计算策略收益
    def evaluate_return_T_p1(self, t_exec, strategy_array_name="default_st"):
        strategy_ = np.array(self.readouts[strategy_array_name].tolist()[:-2])
        short_price_T_p1 = np.array([self.target_raw.loc[day]["Close"].iloc[t_exec]
                                     for day in self.dates[1:-1]])
        long_price_T_p2 = np.array([self.target_raw.loc[day]["Close"].iloc[t_exec]
                                    for day in self.dates[2:]])
        return_rate_T_p1 = pd.Series(
            ((long_price_T_p2 / short_price_T_p1) - 1.) * strategy_,
            index=self.dates[:-2]
        )
        self.readouts["return_" + strategy_array_name + "_" + str(t_exec)] = return_rate_T_p1

    # 夏普比率
    def get_sharpe_ratio(self, strategy_return):
        return_arr = self.readouts[strategy_return].dropna().tolist()[:-2]
        if len(return_arr) == 0 or np.std(return_arr) == 0:
            return 0.0
        sharpe_ratio = np.sqrt(250.) * np.mean(return_arr) / np.std(return_arr)
        return sharpe_ratio

    # 敏感性分析图
    def generate_time_dependency(self, strategy_array_name_list):
        fibonnacci = [0, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 239]
        plt.figure(figsize=(10, 6))
        for strategy_array_name in strategy_array_name_list:
            return_arr = []
            for t_exec in fibonnacci:
                self.evaluate_return_T_p1(t_exec, strategy_array_name=strategy_array_name)
                return_arr.append(
                    self.get_sharpe_ratio("return_" + strategy_array_name + "_" + str(t_exec))
                )
            plt.plot(fibonnacci, return_arr, marker="o", label=strategy_array_name)
        plt.xlabel("trading time (min)")
        plt.ylabel("sharpe ratio")
        plt.title("Time Sensitivity of Strategies")
        plt.legend()
        plt.grid(True)
        plt.show()

    # 调参函数：粗搜+精搜，分别绘图
    def tune_ma_length(self,
                       ma_range=None,
                       t_exec=59,
                       adaptive_window=7,
                       coarse_step=5,
                       refine_radius=10,
                       min_periods_ma_factor=0.1):

        deviation = (self.readouts["T_last_close"] - self.readouts["T_vap"]) / self.readouts["T_vap"]
        rolling_std = deviation.rolling(window=adaptive_window, min_periods=1).std().fillna(method="bfill")

        def sharpe_for_ma(ma_len):
            min_periods_ma = max(1, int(ma_len * min_periods_ma_factor))
            ma = self.readouts["T_last_close"].rolling(window=ma_len,
                                                       min_periods=min_periods_ma).mean().fillna(method="bfill")

            adaptive_st = np.where(deviation > rolling_std, -1,
                                   np.where(deviation < -rolling_std, 1, 0))

            close = self.readouts["T_last_close"].values
            trend_sig = np.zeros_like(adaptive_st, dtype=int)
            for i in range(len(adaptive_st)):
                sig = int(adaptive_st[i])
                if sig == 0:
                    trend_sig[i] = 0
                else:
                    if sig == 1 and close[i] > ma.iat[i]:
                        trend_sig[i] = 1
                    elif sig == -1 and close[i] < ma.iat[i]:
                        trend_sig[i] = -1
                    else:
                        trend_sig[i] = 0

            temp_col = f"_tmp_adaptive_trend_ma{ma_len}"
            self.readouts[temp_col] = trend_sig
            self.evaluate_return_T_p1(t_exec, strategy_array_name=temp_col)
            ret_col = "return_" + temp_col + "_" + str(t_exec)

            if ret_col not in self.readouts.columns:
                sharpe = 0.0
            else:
                returns = self.readouts[ret_col].dropna().tolist()[:-2]
                returns = np.array(returns, dtype=float)
                sharpe = 0.0 if returns.size == 0 or np.std(returns) == 0 else np.sqrt(250.) * np.mean(returns) / np.std(returns)

            trade_count = int((self.readouts[temp_col] != 0).sum())

            try:
                self.readouts.drop(columns=[temp_col, ret_col], inplace=True)
            except Exception:
                pass

            return sharpe, trade_count

        if ma_range is None:
            ma_min, ma_max = 5, 200
            ma_range_full = list(range(ma_min, ma_max + 1))
        else:
            ma_range_full = list(ma_range)

        # 粗搜
        coarse_candidates = list(range(min(ma_range_full), max(ma_range_full) + 1, coarse_step))
        coarse_results = [(m, *sharpe_for_ma(m)) for m in coarse_candidates]
        coarse_df = pd.DataFrame(coarse_results, columns=["MA", "Sharpe", "TradeCount"])
        best_coarse = coarse_df.sort_values("Sharpe", ascending=False).iloc[0]["MA"]

        plt.figure(figsize=(10, 5))
        plt.plot(coarse_df["MA"], coarse_df["Sharpe"], marker="o", label="Coarse Search")
        plt.xlabel("MA Length")
        plt.ylabel("Sharpe Ratio")
        plt.title("Coarse Search: Sharpe vs MA Length")
        plt.legend()
        plt.grid(True)
        plt.show()

        # 精搜
        lo = max(min(ma_range_full), int(best_coarse) - refine_radius)
        hi = min(max(ma_range_full), int(best_coarse) + refine_radius)
        refine_candidates = list(range(lo, hi + 1))
        refine_results = [(m, *sharpe_for_ma(m)) for m in refine_candidates]
        refine_df = pd.DataFrame(refine_results, columns=["MA", "Sharpe", "TradeCount"])

        plt.figure(figsize=(10, 5))
        plt.plot(refine_df["MA"], refine_df["Sharpe"], marker="o", color="orange", label="Refine Search")
        plt.xlabel("MA Length")
        plt.ylabel("Sharpe Ratio")
        plt.title(f"Refine Search around MA={best_coarse}")
        plt.legend()
        plt.grid(True)
        plt.show()

        result_df = pd.concat([coarse_df, refine_df]).drop_duplicates("MA")
        result_df = result_df.sort_values("Sharpe", ascending=False).reset_index(drop=True)
        return result_df


# 使用示例
mydata = stock_info("MinutesIdx.h5", "IC")
# 对比四种策略的敏感性
mydata.generate_time_dependency(["default_st", "improved_st", "adaptive_st", "adaptive_trend_st"])
# 调参寻找最佳均线长度
res = mydata.tune_ma_length(t_exec=59, adaptive_window=7, coarse_step=5, refine_radius=10)
print(res.head(20))

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import seaborn as sns

# 定义类
class stock_info:
    def __init__(self, filepath, target_col):
        # 读取HDF文件
        self.raw = pd.read_hdf(filepath)
        # 提取某一个目标列(如“IC”)
        self.target_raw = self.raw[target_col]
        # 拿到所有的交易日列表
        self.dates = self.target_raw.index.tolist()
        # 初始化一个DataFrame,用于记录结果
        self.readouts = pd.DataFrame(self.dates, columns=["dates"])
        self.readouts.set_index("dates", inplace=True)

        # 计算T日收盘价和VWAP(成交量加权平均价)
        self.readouts["T_last_close"] = [self.target_raw.loc[date]["Close"].iloc[-1] for date in self.dates]
        self.readouts["T_vap"] = [
            (self.target_raw.loc[date]["Close"] * self.target_raw.loc[date]["Volume"]).sum()
            / self.target_raw.loc[date]["Volume"].sum()
            for date in self.dates
        ]

        # --- 策略1: 默认策略 ---
        ratio = np.array(self.readouts["T_last_close"]) / np.array(self.readouts["T_vap"])
        self.readouts["default_st"] = np.where(ratio > 1., -1, 1)

        # --- 策略2: 固定阈值改进 ---
        threshold = 0.002  # 固定阈值 0.2%
        deviation = (self.readouts["T_last_close"] - self.readouts["T_vap"]) / self.readouts["T_vap"]
        self.readouts["improved_st"] = np.where(
            deviation > threshold, -1,
            np.where(deviation < -threshold, 1, 0)
        )

        # --- 策略3: 自适应阈值 ---
        window = 7  # 默认窗口
        rolling_std = deviation.rolling(window=window, min_periods=5).std().fillna(method="bfill")
        self.readouts["adaptive_st"] = np.where(
            deviation > rolling_std, -1,
            np.where(deviation < -rolling_std, 1, 0)
        )

        # --- 策略4: 自适应阈值 + 趋势过滤 ---
        ma_window = 20  # 移动均线长度
        self.readouts["ma_trend"] = self.readouts["T_last_close"].rolling(ma_window).mean()
        self.readouts["adaptive_trend_st"] = np.where(
            (deviation > rolling_std) & (self.readouts["T_last_close"] < self.readouts["ma_trend"]), -1,
            np.where((deviation < -rolling_std) & (self.readouts["T_last_close"] > self.readouts["ma_trend"]), 1, 0)
        )

    # 计算策略收益
    def evaluate_return_T_p1(self, t_exec, strategy_array_name="default_st"):
        strategy_ = np.array(self.readouts[strategy_array_name].tolist()[:-2])
        short_price_T_p1 = np.array([self.target_raw.loc[day]["Close"].iloc[t_exec] for day in self.dates[1:-1]])
        long_price_T_p2 = np.array([self.target_raw.loc[day]["Close"].iloc[t_exec] for day in self.dates[2:]])
        return_rate_T_p1 = pd.Series(((long_price_T_p2 / short_price_T_p1) - 1.) * strategy_, index=self.dates[:-2])
        self.readouts["return_" + strategy_array_name + "_" + str(t_exec)] = return_rate_T_p1

    # 夏普比率
    def get_sharpe_ratio(self, strategy_return):
        return_arr = self.readouts[strategy_return].tolist()[:-2]
        sharpe_ratio = np.sqrt(250.) * np.mean(return_arr) / np.std(return_arr)
        return sharpe_ratio

    # 敏感性分析
    def generate_time_dependency(self, strategy_array_name):
        fig, ax1 = plt.subplots(1, 1)
        fibonnacci = [0, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 239]
        return_arr = []
        for t_exec in fibonnacci:
            self.evaluate_return_T_p1(t_exec, strategy_array_name=strategy_array_name)
            return_arr.append(self.get_sharpe_ratio(strategy_return="return_" + strategy_array_name + "_" + str(t_exec)))
        ax1.plot(fibonnacci, return_arr, label=strategy_array_name)
        ax1.scatter(fibonnacci, return_arr)
        ax1.set_title("time sensitivity of " + strategy_array_name)
        ax1.set_xlabel("trading time (min)")
        ax1.set_ylabel("sharpe ratio")
        plt.legend()
        plt.show()

    # 对比不同策略
    def compare_strategies(self, strategy_list, t_exec=59):
        sharpe_dict = {}
        for st in strategy_list:
            self.evaluate_return_T_p1(t_exec, strategy_array_name=st)
            sharpe_dict[st] = self.get_sharpe_ratio("return_" + st + "_" + str(t_exec))

        sharpe_series = pd.Series(sharpe_dict)
        sharpe_series.plot(kind="bar", title="Sharpe Ratio Comparison (t_exec=" + str(t_exec) + ")")
        plt.ylabel("Sharpe Ratio")
        plt.show()
        return sharpe_series

    # 调参自适应阈值窗口
    def tune_adaptive_window(self, window_range=range(5, 61)):
        best_window = None
        best_sharpe = -np.inf
        sharpe_results = {}

        deviation = (self.readouts["T_last_close"] - self.readouts["T_vap"]) / self.readouts["T_vap"]

        for w in window_range:
            rolling_std = deviation.rolling(window=w, min_periods=5).std().fillna(method="bfill")
            strategy_name = f"adaptive_w{w}"
            self.readouts[strategy_name] = np.where(
                deviation > rolling_std, -1,
                np.where(deviation < -rolling_std, 1, 0)
            )

            self.evaluate_return_T_p1(59, strategy_array_name=strategy_name)
            sharpe = self.get_sharpe_ratio(strategy_return=f"return_{strategy_name}_59")
            sharpe_results[w] = sharpe

            if sharpe > best_sharpe:
                best_sharpe = sharpe
                best_window = w

        # 可视化结果
        plt.plot(list(sharpe_results.keys()), list(sharpe_results.values()), marker="o")
        plt.xlabel("Adaptive Window Size (days)")
        plt.ylabel("Sharpe Ratio")
        plt.title("Sharpe Ratio vs Adaptive Window Size")
        plt.show()

        return best_window, best_sharpe, sharpe_results


# 使用示例
mydata = stock_info("MinutesIdx.h5", "IC")

# 对比四种策略
sharpe_table = mydata.compare_strategies(["default_st", "improved_st", "adaptive_st", "adaptive_trend_st"])
print("四种策略夏普比率：\n", sharpe_table)

# 调参自适应阈值窗口
best_window, best_sharpe, sharpe_results = mydata.tune_adaptive_window(range(5, 61))
print("最佳窗口:", best_window)
print("对应夏普比率:", best_sharpe)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import seaborn as sns

# 定义类
class stock_info:
    def __init__(self, filepath, target_col):
        # 读取HDF文件
        self.raw = pd.read_hdf(filepath)
        # 提取某一个目标列(如“IC”)
        self.target_raw = self.raw[target_col]
        # 拿到所有的交易日列表
        self.dates = self.target_raw.index.tolist()
        # 初始化一个DataFrame,用于记录结果
        self.readouts = pd.DataFrame(self.dates, columns=["dates"])
        self.readouts.set_index("dates", inplace=True)

        # 计算T日收盘价和VWAP(成交量加权平均价)
        self.readouts["T_last_close"] = [self.target_raw.loc[date]["Close"].iloc[-1] for date in self.dates]
        self.readouts["T_vap"] = [
            (self.target_raw.loc[date]["Close"] * self.target_raw.loc[date]["Volume"]).sum()
            / self.target_raw.loc[date]["Volume"].sum()
            for date in self.dates
        ]

        # --- 策略1: 默认策略 ---
        ratio = np.array(self.readouts["T_last_close"]) / np.array(self.readouts["T_vap"])
        self.readouts["default_st"] = np.where(ratio > 1., -1, 1)

        # --- 策略2: 固定阈值改进 ---
        threshold = 0.002  # 固定阈值 0.2%
        deviation = (self.readouts["T_last_close"] - self.readouts["T_vap"]) / self.readouts["T_vap"]
        self.readouts["improved_st"] = np.where(
            deviation > threshold, -1,
            np.where(deviation < -threshold, 1, 0)
        )

        # --- 策略3: 自适应阈值 ---
        window = 7  # 默认窗口
        rolling_std = deviation.rolling(window=window, min_periods=5).std().fillna(method="bfill")
        self.readouts["adaptive_st"] = np.where(
            deviation > rolling_std, -1,
            np.where(deviation < -rolling_std, 1, 0)
        )

        # --- 策略4: 自适应阈值 + 趋势过滤 ---
        ma_window = 20  # 移动均线长度
        self.readouts["ma_trend"] = self.readouts["T_last_close"].rolling(ma_window).mean()
        self.readouts["adaptive_trend_st"] = np.where(
            (deviation > rolling_std) & (self.readouts["T_last_close"] < self.readouts["ma_trend"]), -1,
            np.where((deviation < -rolling_std) & (self.readouts["T_last_close"] > self.readouts["ma_trend"]), 1, 0)
        )

    # 计算策略收益
    def evaluate_return_T_p1(self, t_exec, strategy_array_name="default_st"):
        strategy_ = np.array(self.readouts[strategy_array_name].tolist()[:-2])
        short_price_T_p1 = np.array([self.target_raw.loc[day]["Close"].iloc[t_exec] for day in self.dates[1:-1]])
        long_price_T_p2 = np.array([self.target_raw.loc[day]["Close"].iloc[t_exec] for day in self.dates[2:]])
        return_rate_T_p1 = pd.Series(((long_price_T_p2 / short_price_T_p1) - 1.) * strategy_, index=self.dates[:-2])
        self.readouts["return_" + strategy_array_name + "_" + str(t_exec)] = return_rate_T_p1

    # 夏普比率
    def get_sharpe_ratio(self, strategy_return):
        return_arr = self.readouts[strategy_return].tolist()[:-2]
        sharpe_ratio = np.sqrt(250.) * np.mean(return_arr) / np.std(return_arr)
        return sharpe_ratio

    # 敏感性分析
    def generate_time_dependency(self, strategy_array_name):
        fig, ax1 = plt.subplots(1, 1)
        fibonnacci = [0, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 239]
        return_arr = []
        for t_exec in fibonnacci:
            self.evaluate_return_T_p1(t_exec, strategy_array_name=strategy_array_name)
            return_arr.append(self.get_sharpe_ratio(strategy_return="return_" + strategy_array_name + "_" + str(t_exec)))
        ax1.plot(fibonnacci, return_arr, label=strategy_array_name)
        ax1.scatter(fibonnacci, return_arr)
        ax1.set_title("time sensitivity of " + strategy_array_name)
        ax1.set_xlabel("trading time (min)")
        ax1.set_ylabel("sharpe ratio")
        plt.legend()
        plt.show()

    # 对比不同策略
    def compare_strategies(self, strategy_list, t_exec=59):
        sharpe_dict = {}
        for st in strategy_list:
            self.evaluate_return_T_p1(t_exec, strategy_array_name=st)
            sharpe_dict[st] = self.get_sharpe_ratio("return_" + st + "_" + str(t_exec))

        sharpe_series = pd.Series(sharpe_dict)
        sharpe_series.plot(kind="bar", title="Sharpe Ratio Comparison (t_exec=" + str(t_exec) + ")")
        plt.ylabel("Sharpe Ratio")
        plt.show()
        return sharpe_series

    # 调参固定阈值
    def tune_fixed_threshold(self, thresholds=np.linspace(0.001, 0.01, 20)):
        best_th = None
        best_sharpe = -np.inf
        sharpe_results = {}

        deviation = (self.readouts["T_last_close"] - self.readouts["T_vap"]) / self.readouts["T_vap"]

        for th in thresholds:
            strategy_name = f"fixed_th{th:.4f}"
            self.readouts[strategy_name] = np.where(
                deviation > th, -1,
                np.where(deviation < -th, 1, 0)
            )
            self.evaluate_return_T_p1(59, strategy_array_name=strategy_name)
            sharpe = self.get_sharpe_ratio(strategy_return=f"return_{strategy_name}_59")
            sharpe_results[th] = sharpe

            if sharpe > best_sharpe:
                best_sharpe = sharpe
                best_th = th

        # 可视化
        plt.plot(list(sharpe_results.keys()), list(sharpe_results.values()), marker="o")
        plt.xlabel("Fixed Threshold")
        plt.ylabel("Sharpe Ratio")
        plt.title("Sharpe Ratio vs Fixed Threshold")
        plt.show()

        return best_th, best_sharpe, sharpe_results

    # 调参自适应阈值窗口
    def tune_adaptive_window(self, window_range=range(5, 61)):
        best_window = None
        best_sharpe = -np.inf
        sharpe_results = {}

        deviation = (self.readouts["T_last_close"] - self.readouts["T_vap"]) / self.readouts["T_vap"]

        for w in window_range:
            rolling_std = deviation.rolling(window=w, min_periods=5).std().fillna(method="bfill")
            strategy_name = f"adaptive_w{w}"
            self.readouts[strategy_name] = np.where(
                deviation > rolling_std, -1,
                np.where(deviation < -rolling_std, 1, 0)
            )

            self.evaluate_return_T_p1(59, strategy_array_name=strategy_name)
            sharpe = self.get_sharpe_ratio(strategy_return=f"return_{strategy_name}_59")
            sharpe_results[w] = sharpe

            if sharpe > best_sharpe:
                best_sharpe = sharpe
                best_window = w

        # 可视化结果
        plt.plot(list(sharpe_results.keys()), list(sharpe_results.values()), marker="o")
        plt.xlabel("Adaptive Window Size (days)")
        plt.ylabel("Sharpe Ratio")
        plt.title("Sharpe Ratio vs Adaptive Window Size")
        plt.show()

        return best_window, best_sharpe, sharpe_results


# 使用示例
mydata = stock_info("MinutesIdx.h5", "IC")

# 对比四种策略
sharpe_table = mydata.compare_strategies(["default_st", "improved_st", "adaptive_st", "adaptive_trend_st"])
print("四种策略夏普比率：\n", sharpe_table)

# 调参固定阈值
best_th, best_sharpe_th, sharpe_results_th = mydata.tune_fixed_threshold()
print("最佳固定阈值:", best_th)
print("对应夏普比率:", best_sharpe_th)

# 调参自适应阈值窗口
best_window, best_sharpe_w, sharpe_results_w = mydata.tune_adaptive_window()
print("最佳自适应窗口:", best_window)
print("对应夏普比率:", best_sharpe_w)

### 动态仓位控制与前四种方式的对比

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

class stock_info:
    def __init__(self, filepath, target_col):
        # 添加文件存在性检查
        if not os.path.exists(filepath):
            raise FileNotFoundError(f"文件 {filepath} 不存在，请检查文件路径是否正确")
        
        try:
            self.raw = pd.read_hdf(filepath)
        except Exception as e:
            raise Exception(f"读取HDF文件失败: {e}")
        
        # 添加数据结构检查
        if target_col not in self.raw.columns and target_col not in self.raw.keys():
            raise ValueError(f"目标列 '{target_col}' 不存在于文件中。可用的列/键: {list(self.raw.columns) if hasattr(self.raw, 'columns') else list(self.raw.keys())}")
        
        try:
            self.target_raw = self.raw[target_col]
        except Exception as e:
            raise Exception(f"访问目标数据 '{target_col}' 失败: {e}")

        # 改进的数据结构处理逻辑
        print(f"target_raw 类型: {type(self.target_raw)}")
        print(f"target_raw 形状: {self.target_raw.shape if hasattr(self.target_raw, 'shape') else 'N/A'}")
        
        # 处理 Series 中的复杂数据结构
        if isinstance(self.target_raw, pd.Series):
            # 检查 Series 中的第一个元素类型
            if len(self.target_raw) > 0:
                first_element = self.target_raw.iloc[0]
                print(f"第一个元素类型: {type(first_element)}")
                
                # 如果元素是 DataFrame 或 Series，需要特殊处理
                if isinstance(first_element, (pd.DataFrame, pd.Series)):
                    print("检测到嵌套数据结构，提取收盘价...")
                    # 假设每个元素是一个包含 'Close' 列的 DataFrame
                    try:
                        # 尝试提取每个元素的 'Close' 值
                        close_values = []
                        dates = []
                        for date, item in self.target_raw.items():
                            if isinstance(item, pd.DataFrame) and 'Close' in item.columns:
                                # 取最后一个收盘价（当天最后一分钟）
                                close_values.append(item['Close'].iloc[-1])
                            elif isinstance(item, pd.Series):
                                # 如果是 Series，取最后一个值
                                close_values.append(item.iloc[-1])
                            else:
                                # 如果是数值，直接使用
                                close_values.append(item)
                            dates.append(date)
                        
                        self.price = pd.Series(close_values, index=dates).astype(float)
                    except Exception as e:
                        print(f"嵌套数据处理失败: {e}")
                        # 降级处理：尝试将整个 Series 转换为 float
                        self.price = self.target_raw.astype(float)
                else:
                    # 直接转换为 float
                    try:
                        self.price = self.target_raw.astype(float)
                    except Exception as e:
                        print(f"直接转换为 float 失败: {e}")
                        # 如果转换失败，尝试其他方法
                        self.price = pd.Series([float(x) if not isinstance(x, (list, np.ndarray)) 
                                              else float(x[-1]) for x in self.target_raw], 
                                             index=self.target_raw.index)
            else:
                raise ValueError("目标数据为空")
        else:
            # 如果 target_raw 是 DataFrame
            if isinstance(self.target_raw, pd.DataFrame):
                if "Close" in self.target_raw.columns:
                    self.price = self.target_raw["Close"].astype(float)
                else:
                    # 如果 DataFrame 没有 'Close' 列，使用第一列数值数据
                    numeric_cols = self.target_raw.select_dtypes(include=[np.number]).columns
                    if len(numeric_cols) > 0:
                        self.price = self.target_raw[numeric_cols[0]].astype(float)
                    else:
                        raise ValueError(f"'{target_col}' 是DataFrame但不包含数值型列。可用列: {list(self.target_raw.columns)}")
            else:
                # 其他情况，直接转换
                self.price = self.target_raw.astype(float)

        print(f"price 类型: {type(self.price)}")
        print(f"price 形状: {self.price.shape if hasattr(self.price, 'shape') else 'N/A'}")
        print(f"price 索引类型: {type(self.price.index)}")
        print(f"price 前5个值: {self.price.head()}")

        self.trading_days = self.price.index

    def calculate_sharpe(self, pnl):
        if len(pnl) == 0:
            return 0
        ret = pnl.pct_change().dropna()
        if len(ret) == 0:
            return 0
        if ret.std() == 0 or np.isnan(ret.std()) or ret.std() == np.inf:
            return 0
        # 检查是否所有收益都相同
        if len(ret) > 1 and np.allclose(ret.values, ret.iloc[0]):
            return 0
        # 按照之前代码的方法计算夏普比率，使用252作为年度化因子
        sharpe = np.sqrt(252) * ret.mean() / ret.std()
        return sharpe if not np.isnan(sharpe) and np.isfinite(sharpe) else 0

    # === 策略1：固定阈值 ===
    def strategy_fixed_threshold(self, window=20, threshold=1.5):
        if len(self.price) < window:
            raise ValueError(f"数据长度 ({len(self.price)}) 小于窗口大小 ({window})")
            
        rolling_mean = self.price.rolling(window).mean()
        rolling_std = self.price.rolling(window).std()
        
        # 处理标准差为0的情况
        zscore = (self.price - rolling_mean) / rolling_std.replace(0, np.nan)
        zscore = zscore.fillna(0)

        position = np.where(zscore > threshold, -1,
                   np.where(zscore < -threshold, 1, 0))

        pnl = pd.Series(position, index=self.price.index).shift(1) * self.price.pct_change()
        # 处理可能的NaN值
        pnl = pnl.fillna(0)
        
        cumulative_pnl = pnl.cumsum()
        sharpe_ratio = self.calculate_sharpe(cumulative_pnl)
        return cumulative_pnl, sharpe_ratio

    # === 策略2：改进阈值（双阈值带宽） ===
    def strategy_improved_threshold(self, window=20, upper=1.5, lower=0.5):
        if len(self.price) < window:
            raise ValueError(f"数据长度 ({len(self.price)}) 小于窗口大小 ({window})")
            
        rolling_mean = self.price.rolling(window).mean()
        rolling_std = self.price.rolling(window).std()
        
        # 处理标准差为0的情况
        zscore = (self.price - rolling_mean) / rolling_std.replace(0, np.nan)
        zscore = zscore.fillna(0)

        position = np.zeros(len(zscore))
        pos = 0
        for i in range(len(zscore)):
            if zscore.iloc[i] > upper:
                pos = -1
            elif zscore.iloc[i] < -upper:
                pos = 1
            elif abs(zscore.iloc[i]) < lower:
                pos = 0
            position[i] = pos

        pnl = pd.Series(position, index=self.price.index).shift(1) * self.price.pct_change()
        # 处理可能的NaN值
        pnl = pnl.fillna(0)
        
        cumulative_pnl = pnl.cumsum()
        sharpe_ratio = self.calculate_sharpe(cumulative_pnl)
        return cumulative_pnl, sharpe_ratio

    # === 策略3：自适应阈值 ===
    def strategy_adaptive_threshold(self, window=20, adapt_window=7, scale=1.5):
        if len(self.price) < max(window, adapt_window):
            raise ValueError(f"数据长度 ({len(self.price)}) 小于所需窗口大小 (max({window}, {adapt_window}))")
            
        rolling_mean = self.price.rolling(window).mean()
        rolling_std = self.price.rolling(window).std()
        
        # 处理标准差为0的情况
        zscore = (self.price - rolling_mean) / rolling_std.replace(0, np.nan)
        zscore = zscore.fillna(0)

        adaptive_threshold = scale * zscore.rolling(adapt_window).std().fillna(0)

        position = np.where(zscore > adaptive_threshold, -1,
                   np.where(zscore < -adaptive_threshold, 1, 0))

        pnl = pd.Series(position, index=self.price.index).shift(1) * self.price.pct_change()
        # 处理可能的NaN值
        pnl = pnl.fillna(0)
        
        cumulative_pnl = pnl.cumsum()
        sharpe_ratio = self.calculate_sharpe(cumulative_pnl)
        return cumulative_pnl, sharpe_ratio

    # === 策略4：自适应阈值 + 趋势过滤 ===
    def strategy_adaptive_trend(self, window=20, adapt_window=7, scale=1.5, ma_len=50):
        if len(self.price) < max(window, adapt_window, ma_len):
            raise ValueError(f"数据长度 ({len(self.price)}) 小于所需窗口大小 (max({window}, {adapt_window}, {ma_len}))")
            
        rolling_mean = self.price.rolling(window).mean()
        rolling_std = self.price.rolling(window).std()
        
        # 处理标准差为0的情况
        zscore = (self.price - rolling_mean) / rolling_std.replace(0, np.nan)
        zscore = zscore.fillna(0)

        adaptive_threshold = scale * zscore.rolling(adapt_window).std().fillna(0)

        position = np.where(zscore > adaptive_threshold, -1,
                   np.where(zscore < -adaptive_threshold, 1, 0))

        # 趋势过滤：用均线确认大趋势
        trend = self.price.rolling(ma_len).mean()
        trend_signal = np.where(self.price > trend, 1, -1)

        position = position * trend_signal

        pnl = pd.Series(position, index=self.price.index).shift(1) * self.price.pct_change()
        # 处理可能的NaN值
        pnl = pnl.fillna(0)
        
        cumulative_pnl = pnl.cumsum()
        sharpe_ratio = self.calculate_sharpe(cumulative_pnl)
        return cumulative_pnl, sharpe_ratio

    # === 策略5：动态仓位控制 ===
    def strategy_dynamic_position(self, window=20, threshold=1.5):
        if len(self.price) < window:
            raise ValueError(f"数据长度 ({len(self.price)}) 小于窗口大小 ({window})")
            
        rolling_mean = self.price.rolling(window).mean()
        rolling_std = self.price.rolling(window).std()
        
        # 处理标准差为0的情况
        zscore = (self.price - rolling_mean) / rolling_std.replace(0, np.nan)
        zscore = zscore.fillna(0)

        # 仓位强度 = tanh(zscore/threshold)，范围 [-1,1]
        position = np.tanh(zscore / threshold)

        pnl = pd.Series(position, index=self.price.index).shift(1) * self.price.pct_change()
        # 处理可能的NaN值
        pnl = pnl.fillna(0)
        
        cumulative_pnl = pnl.cumsum()
        sharpe_ratio = self.calculate_sharpe(cumulative_pnl)
        return cumulative_pnl, sharpe_ratio

    # === 策略6：波动率加权策略 ===
    def strategy_volatility_weighted(self, window=20, threshold=1.5, vol_window=10, max_position=2.0):
        """
        波动率加权策略：
        - 基础信号：z-score偏离度
        - 波动率调整：根据历史波动率调整仓位大小
        - 仓位限制：设置最大仓位避免过度杠杆
        """
        if len(self.price) < max(window, vol_window):
            raise ValueError(f"数据长度 ({len(self.price)}) 小于所需窗口大小 (max({window}, {vol_window}))")
            
        # 计算基础z-score信号
        rolling_mean = self.price.rolling(window).mean()
        rolling_std = self.price.rolling(window).std()
        zscore = (self.price - rolling_mean) / rolling_std.replace(0, np.nan)
        zscore = zscore.fillna(0)
        
        # 计算历史波动率（使用对数收益率）
        log_returns = np.log(self.price / self.price.shift(1)).fillna(0)
        volatility = log_returns.rolling(vol_window).std().fillna(0) * np.sqrt(252)  # 年化波动率
        
        # 计算平均波动率作为基准
        avg_volatility = volatility.rolling(60).mean().fillna(volatility.mean())
        
        # 波动率调整因子：低波动时增加仓位，高波动时减少仓位
        # 使用inverse volatility scaling
        vol_weight = avg_volatility / (volatility + 1e-8)  # 避免除零
        vol_weight = vol_weight.clip(0.1, 3.0)  # 限制调整范围
        
        # 基础信号方向
        base_signal = np.where(zscore > threshold, -1,
                      np.where(zscore < -threshold, 1, 0))
        
        # 应用波动率权重
        weighted_position = base_signal * vol_weight
        
        # 应用最大仓位限制
        weighted_position = np.clip(weighted_position, -max_position, max_position)
        
        # 计算PnL
        pnl = pd.Series(weighted_position, index=self.price.index).shift(1) * self.price.pct_change()
        pnl = pnl.fillna(0)
        
        cumulative_pnl = pnl.cumsum()
        sharpe_ratio = self.calculate_sharpe(cumulative_pnl)
        
        return cumulative_pnl, sharpe_ratio

    # === 策略7：波动率加权 + 动态阈值 ===
    def strategy_vol_weighted_dynamic_threshold(self, window=20, base_threshold=1.5, 
                                               vol_window=10, max_position=2.0):
        """
        波动率加权 + 动态阈值策略：
        - 动态阈值：根据市场波动率调整交易阈值
        - 波动率加权：调整仓位大小
        - 双重适应机制
        """
        if len(self.price) < max(window, vol_window):
            raise ValueError(f"数据长度 ({len(self.price)}) 小于所需窗口大小 (max({window}, {vol_window}))")
            
        # 计算基础指标
        rolling_mean = self.price.rolling(window).mean()
        rolling_std = self.price.rolling(window).std()
        zscore = (self.price - rolling_mean) / rolling_std.replace(0, np.nan)
        zscore = zscore.fillna(0)
        
        # 计算波动率
        log_returns = np.log(self.price / self.price.shift(1)).fillna(0)
        volatility = log_returns.rolling(vol_window).std().fillna(0) * np.sqrt(252)
        avg_volatility = volatility.rolling(60).mean().fillna(volatility.mean())
        
        # 动态阈值：高波动时提高阈值，低波动时降低阈值
        vol_ratio = volatility / (avg_volatility + 1e-8)
        dynamic_threshold = base_threshold * vol_ratio
        dynamic_threshold = dynamic_threshold.clip(0.5, 3.0)  # 限制阈值范围
        
        # 基础信号（使用动态阈值）
        base_signal = np.where(zscore > dynamic_threshold, -1,
                      np.where(zscore < -dynamic_threshold, 1, 0))
        
        # 波动率权重
        vol_weight = avg_volatility / (volatility + 1e-8)
        vol_weight = vol_weight.clip(0.1, 3.0)
        
        # 应用权重和仓位限制
        weighted_position = base_signal * vol_weight
        weighted_position = np.clip(weighted_position, -max_position, max_position)
        
        # 计算PnL
        pnl = pd.Series(weighted_position, index=self.price.index).shift(1) * self.price.pct_change()
        pnl = pnl.fillna(0)
        
        cumulative_pnl = pnl.cumsum()
        sharpe_ratio = self.calculate_sharpe(cumulative_pnl)
        
        return cumulative_pnl, sharpe_ratio

    # === 策略8：多时间框架策略 ===
    def strategy_multi_timeframe(self, short_window=5, medium_window=20, long_window=60, 
                                threshold=1.5, weight_short=0.5, weight_medium=0.3, weight_long=0.2):
        """
        多时间框架策略：
        - 短期(5分钟)：捕捉快速反转
        - 中期(20分钟)：平衡噪音和趋势
        - 长期(60分钟)：确认大趋势
        - 加权组合：根据可靠性分配权重
        """
        required_window = max(short_window, medium_window, long_window)
        if len(self.price) < required_window:
            raise ValueError(f"数据长度 ({len(self.price)}) 小于所需窗口大小 ({required_window})")
        
        # 计算不同时间框架的z-score
        def calculate_zscore(window):
            rolling_mean = self.price.rolling(window).mean()
            rolling_std = self.price.rolling(window).std()
            zscore = (self.price - rolling_mean) / rolling_std.replace(0, np.nan)
            return zscore.fillna(0)
        
        zscore_short = calculate_zscore(short_window)
        zscore_medium = calculate_zscore(medium_window)
        zscore_long = calculate_zscore(long_window)
        
        # 生成各时间框架的信号
        def generate_signal(zscore, thresh):
            return np.where(zscore > thresh, -1,
                   np.where(zscore < -thresh, 1, 0))
        
        signal_short = generate_signal(zscore_short, threshold)
        signal_medium = generate_signal(zscore_medium, threshold * 1.2)  # 中期稍高阈值
        signal_long = generate_signal(zscore_long, threshold * 1.5)    # 长期更高阈值
        
        # 趋势过滤（使用长期信号）
        trend_filter = np.where(self.price > self.price.rolling(long_window).mean(), 1, -1)
        
        # 加权组合信号（考虑趋势）
        combined_signal = (
            weight_short * signal_short * trend_filter +
            weight_medium * signal_medium * trend_filter +
            weight_long * signal_long
        )
        
        # 信号强度归一化到[-1,1]
        signal_strength = np.tanh(combined_signal)
        
        # 计算PnL
        pnl = pd.Series(signal_strength, index=self.price.index).shift(1) * self.price.pct_change()
        pnl = pnl.fillna(0)
        
        cumulative_pnl = pnl.cumsum()
        sharpe_ratio = self.calculate_sharpe(cumulative_pnl)
        
        return cumulative_pnl, sharpe_ratio

    # === 策略9：机器学习增强策略 ===
    def strategy_ml_enhanced(self, window=20, lookback_period=100, train_test_split=0.7):
        """
        机器学习增强策略：
        - 特征工程：价格动量、波动率、成交量变化等
        - 随机森林分类器：预测价格方向
        - 动态权重：根据预测概率调整仓位
        """
        if len(self.price) < max(window, lookback_period):
            raise ValueError(f"数据长度 ({len(self.price)}) 小于所需窗口大小 (max({window}, {lookback_period}))")
        
        # 特征工程
        def create_features(price):
            features = pd.DataFrame(index=price.index)
            
            # 价格特征
            features['returns'] = price.pct_change().fillna(0)
            features['log_returns'] = np.log(price / price.shift(1)).fillna(0)
            
            # 移动平均特征
            for ma in [5, 10, 20]:
                features[f'ma_{ma}'] = price.rolling(ma).mean()
                features[f'price_ma_{ma}_ratio'] = price / features[f'ma_{ma}']
            
            # 波动率特征
            features['volatility_5'] = features['log_returns'].rolling(5).std().fillna(0)
            features['volatility_20'] = features['log_returns'].rolling(20).std().fillna(0)
            
            # 动量特征
            features['momentum_5'] = price / price.shift(5) - 1
            features['momentum_20'] = price / price.shift(20) - 1
            
            # RSI特征
            def calculate_rsi(prices, period=14):
                delta = prices.diff()
                gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
                loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
                rs = gain / loss
                rsi = 100 - (100 / (1 + rs))
                return rsi.fillna(50)
            
            features['rsi'] = calculate_rsi(price)
            
            # 填充NaN值
            features = features.fillna(method='ffill').fillna(0)
            
            return features
        
        features = create_features(self.price)
        
        # 目标变量：未来1期收益方向
        future_returns = self.price.pct_change().shift(-1).fillna(0)
        target = (future_returns > 0).astype(int)
        
        # 训练测试分割
        split_idx = int(len(features) * train_test_split)
        X_train = features.iloc[:split_idx]
        y_train = target.iloc[:split_idx]
        X_test = features.iloc[split_idx:]
        y_test = target.iloc[split_idx:]
        
        # 训练模型
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        
        rf_model = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=5)
        rf_model.fit(X_train_scaled, y_train)
        
        # 预测概率
        all_features_scaled = scaler.transform(features)
        predict_prob = rf_model.predict_proba(all_features_scaled)[:, 1]  # 上涨概率
        
        # 生成信号：概率偏离0.5时产生信号
        prob_deviation = predict_prob - 0.5
        signal_strength = np.tanh(prob_deviation * 10)  # 放大信号强度
        
        # 计算PnL
        pnl = pd.Series(signal_strength, index=self.price.index).shift(1) * self.price.pct_change()
        pnl = pnl.fillna(0)
        
        cumulative_pnl = pnl.cumsum()
        sharpe_ratio = self.calculate_sharpe(cumulative_pnl)
        
        return cumulative_pnl, sharpe_ratio

    # === 策略10：风险平价策略 ===
    def strategy_risk_parity(self, window=20, vol_window=10, target_vol=0.15, max_leverage=3.0):
        """
        风险平价策略：
        - 目标波动率：控制组合波动在目标水平
        - 动态杠杆：根据实际波动调整杠杆
        - 多重信号源：结合反转和动量信号
        """
        if len(self.price) < max(window, vol_window):
            raise ValueError(f"数据长度 ({len(self.price)}) 小于所需窗口大小 (max({window}, {vol_window}))")
        
        # 计算基础信号
        rolling_mean = self.price.rolling(window).mean()
        rolling_std = self.price.rolling(window).std()
        zscore = (self.price - rolling_mean) / rolling_std.replace(0, np.nan)
        zscore = zscore.fillna(0)
        
        # 反转信号
        reversion_signal = np.where(zscore > 1.5, -1,
                           np.where(zscore < -1.5, 1, 0))
        
        # 动量信号
        momentum_signal = np.where(self.price > self.price.rolling(10).mean(), 1, -1)
        
        # 组合信号
        combined_signal = 0.6 * reversion_signal + 0.4 * momentum_signal
        
        # 计算实际波动率
        log_returns = np.log(self.price / self.price.shift(1)).fillna(0)
        current_vol = log_returns.rolling(vol_window).std().fillna(0.01) * np.sqrt(252)
        
        # 计算目标杠杆以达到目标波动率
        leverage = target_vol / (current_vol + 1e-8)
        leverage = np.clip(leverage, 1/max_leverage, max_leverage)
        
        # 应用杠杆限制
        final_position = combined_signal * leverage
        
        # 计算PnL
        pnl = pd.Series(final_position, index=self.price.index).shift(1) * self.price.pct_change()
        pnl = pnl.fillna(0)
        
        cumulative_pnl = pnl.cumsum()
        sharpe_ratio = self.calculate_sharpe(cumulative_pnl)
        
        return cumulative_pnl, sharpe_ratio

    # === HTML5可视化报告生成器 ===
    def generate_html_report(self, results_dict):
        """生成HTML5格式的策略分析报告"""
        
        # 策略描述信息
        strategy_descriptions = {
            "Fixed Threshold": "基于固定阈值的均值回归策略，简单直接的反转交易",
            "Improved Threshold": "双阈值带宽策略，减少频繁交易，过滤噪音信号",
            "Adaptive Threshold": "自适应阈值策略，根据市场波动动态调整交易阈值",
            "Adaptive Threshold + Trend": "结合趋势过滤的自适应策略，避免逆势交易",
            "Dynamic Position": "动态仓位控制策略，使用tanh函数实现平滑仓位调整",
            "Volatility Weighted": "波动率加权策略，根据市场波动率调整仓位大小",
            "Vol Weighted + Dynamic Threshold": "波动率加权+动态阈值，双重适应机制",
            "Multi Timeframe": "多时间框架策略，结合短期、中期、长期信号",
            "ML Enhanced": "机器学习增强策略，使用随机森林预测价格方向",
            "Risk Parity": "风险平价策略，控制组合波动率在目标水平"
        }
        
        # 策略技术细节
        strategy_details = {
            "Fixed Threshold": "z-score偏离度 | 固定1.5标准差阈值 | 全仓交易",
            "Improved Threshold": "z-score偏离度 | 上下双阈值 | 状态机仓位管理",
            "Adaptive Threshold": "z-score偏离度 | 滚动标准差自适应阈值 | 滤波优化",
            "Adaptive Threshold + Trend": "自适应信号 + 长期均线趋势过滤 | 顺势交易",
            "Dynamic Position": "tanh函数仓位映射 | 连续仓位调整 | 风险平滑",
            "Volatility Weighted": "inverse volatility scaling | 波动率倒数加权 | 仓位自适应",
            "Vol Weighted + Dynamic Threshold": "波动率加权 + 动态阈值 | 双重风险管理",
            "Multi Timeframe": "5/20/60分钟多周期 | 权重组合 | 趋势确认",
            "ML Enhanced": "随机森林分类器 | 多维特征工程 | 概率加权",
            "Risk Parity": "目标波动率控制 | 动态杠杆管理 | 风险预算分配"
        }
        
        # 找出最佳策略
        best_strategy = max(results_dict.items(), key=lambda x: x[1])
        
        # 生成HTML内容
        html_content = f"""
<!DOCTYPE html>
<html lang="zh-CN">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>量化交易策略分析报告</title>
    <style>
        * {{
            margin: 0;
            padding: 0;
            box-sizing: border-box;
        }}
        
        body {{
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            min-height: 100vh;
            padding: 20px;
        }}
        
        .container {{
            max-width: 1200px;
            margin: 0 auto;
            background: white;
            border-radius: 15px;
            box-shadow: 0 20px 40px rgba(0,0,0,0.1);
            overflow: hidden;
        }}
        
        .header {{
            background: linear-gradient(135deg, #2c3e50 0%, #3498db 100%);
            color: white;
            padding: 30px;
            text-align: center;
        }}
        
        .header h1 {{
            font-size: 2.5em;
            margin-bottom: 10px;
            text-shadow: 2px 2px 4px rgba(0,0,0,0.3);
        }}
        
        .header p {{
            font-size: 1.2em;
            opacity: 0.9;
        }}
        
        .best-strategy {{
            background: linear-gradient(135deg, #f39c12 0%, #e74c3c 100%);
            color: white;
            padding: 20px;
            text-align: center;
            font-size: 1.3em;
            font-weight: bold;
        }}
        
        .content {{
            padding: 30px;
        }}
        
        .strategy-grid {{
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(300px, 1fr));
            gap: 20px;
            margin-bottom: 30px;
        }}
        
        .strategy-card {{
            border: 1px solid #e0e0e0;
            border-radius: 10px;
            padding: 20px;
            transition: transform 0.3s ease, box-shadow 0.3s ease;
            background: white;
        }}
        
        .strategy-card:hover {{
            transform: translateY(-5px);
            box-shadow: 0 10px 25px rgba(0,0,0,0.15);
        }}
        
        .strategy-card.best {{
            border: 2px solid #f39c12;
            background: linear-gradient(135deg, #fff9e6 0%, #ffffff 100%);
        }}
        
        .strategy-name {{
            font-size: 1.3em;
            font-weight: bold;
            color: #2c3e50;
            margin-bottom: 10px;
            border-bottom: 2px solid #3498db;
            padding-bottom: 5px;
        }}
        
        .strategy-sharpe {{
            font-size: 2em;
            font-weight: bold;
            text-align: center;
            margin: 15px 0;
            padding: 10px;
            border-radius: 8px;
        }}
        
        .sharpe-excellent {{ background: #27ae60; color: white; }}
        .sharpe-good {{ background: #f39c12; color: white; }}
        .sharpe-average {{ background: #e74c3c; color: white; }}
        .sharpe-poor {{ background: #95a5a6; color: white; }}
        
        .strategy-description {{
            color: #555;
            line-height: 1.6;
            margin-bottom: 15px;
        }}
        
        .strategy-details {{
            background: #f8f9fa;
            padding: 15px;
            border-radius: 8px;
            border-left: 4px solid #3498db;
        }}
        
        .strategy-details h4 {{
            color: #2c3e50;
            margin-bottom: 8px;
        }}
        
        .strategy-details p {{
            color: #666;
            font-size: 0.9em;
            line-height: 1.5;
        }}
        
        .performance-metrics {{
            background: #ecf0f1;
            padding: 20px;
            border-radius: 10px;
            margin-top: 30px;
        }}
        
        .metrics-grid {{
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(200px, 1fr));
            gap: 15px;
        }}
        
        .metric-item {{
            background: white;
            padding: 15px;
            border-radius: 8px;
            text-align: center;
            border: 1px solid #bdc3c7;
        }}
        
        .metric-value {{
            font-size: 1.5em;
            font-weight: bold;
            color: #2c3e50;
        }}
        
        .metric-label {{
            color: #7f8c8d;
            font-size: 0.9em;
            margin-top: 5px;
        }}
        
        .footer {{
            background: #34495e;
            color: white;
            text-align: center;
            padding: 20px;
        }}
        
        @media (max-width: 768px) {{
            .strategy-grid {{
                grid-template-columns: 1fr;
            }}
            
            .header h1 {{
                font-size: 2em;
            }}
            
            .metrics-grid {{
                grid-template-columns: repeat(2, 1fr);
            }}
        }}
    </style>
</head>
<body>
    <div class="container">
        <div class="header">
            <h1>🚀 量化交易策略分析报告</h1>
            <p>基于 {self.price.index[0].strftime('%Y-%m-%d')} 至 {self.price.index[-1].strftime('%Y-%m-%d')} 数据</p>
        </div>
        
        <div class="best-strategy">
            🏆 最佳策略: {best_strategy[0]} | 夏普比率: {best_strategy[1]:.4f}
        </div>
        
        <div class="content">
            <div class="strategy-grid">
"""
        
        # 生成每个策略的卡片
        for strategy_name, sharpe_ratio in results_dict.items():
            is_best = strategy_name == best_strategy[0]
            
            # 根据夏普比率设置等级
            if sharpe_ratio > 2.0:
                sharpe_class = "sharpe-excellent"
                performance_text = "优秀"
            elif sharpe_ratio > 1.0:
                sharpe_class = "sharpe-good"
                performance_text = "良好"
            elif sharpe_ratio > 0.5:
                sharpe_class = "sharpe-average"
                performance_text = "一般"
            else:
                sharpe_class = "sharpe-poor"
                performance_text = "较差"
            
            card_class = "strategy-card best" if is_best else "strategy-card"
            
            html_content += f"""
                <div class="{card_class}">
                    <div class="strategy-name">{strategy_name}</div>
                    <div class="strategy-sharpe {sharpe_class}">
                        {sharpe_ratio:.4f}<br>
                        <small>{performance_text}</small>
                    </div>
                    <div class="strategy-description">
                        {strategy_descriptions.get(strategy_name, "策略描述待补充")}
                    </div>
                    <div class="strategy-details">
                        <h4>🔧 技术特点</h4>
                        <p>{strategy_details.get(strategy_name, "技术细节待补充")}</p>
                    </div>
                </div>
"""
        
        # 计算汇总统计
        all_sharpes = list(results_dict.values())
        avg_sharpe = np.mean(all_sharpes)
        max_sharpe = max(all_sharpes)
        min_sharpe = min(all_sharpes)
        std_sharpe = np.std(all_sharpes)
        positive_count = sum(1 for s in all_sharpes if s > 0)
        
        html_content += f"""
            </div>
            
            <div class="performance-metrics">
                <h3 style="text-align: center; margin-bottom: 20px; color: #2c3e50;">📊 策略表现汇总</h3>
                <div class="metrics-grid">
                    <div class="metric-item">
                        <div class="metric-value">{avg_sharpe:.4f}</div>
                        <div class="metric-label">平均夏普比率</div>
                    </div>
                    <div class="metric-item">
                        <div class="metric-value">{max_sharpe:.4f}</div>
                        <div class="metric-label">最高夏普比率</div>
                    </div>
                    <div class="metric-item">
                        <div class="metric-value">{min_sharpe:.4f}</div>
                        <div class="metric-label">最低夏普比率</div>
                    </div>
                    <div class="metric-item">
                        <div class="metric-value">{std_sharpe:.4f}</div>
                        <div class="metric-label">夏普比率标准差</div>
                    </div>
                    <div class="metric-item">
                        <div class="metric-value">{positive_count}/{len(all_sharpes)}</div>
                        <div class="metric-label">正收益策略数量</div>
                    </div>
                    <div class="metric-item">
                        <div class="metric-value">{len(all_sharpes)}</div>
                        <div class="metric-label">策略总数</div>
                    </div>
                </div>
            </div>
        </div>
        
        <div class="footer">
            <p>📈 本报告由量化交易策略分析系统生成 | {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}</p>
            <p>⚠️ 风险提示：过往业绩不代表未来表现，投资需谨慎</p>
        </div>
    </div>
    
    <script>
        // 添加一些交互效果
        document.addEventListener('DOMContentLoaded', function() {{
            const cards = document.querySelectorAll('.strategy-card');
            
            cards.forEach(card => {{
                card.addEventListener('click', function() {{
                    // 简单的点击高亮效果
                    cards.forEach(c => c.style.transform = 'scale(1)');
                    this.style.transform = 'scale(1.02)';
                }});
            }});
            
            // 添加数字动画效果
            const sharpeElements = document.querySelectorAll('.strategy-sharpe');
            sharpeElements.forEach(element => {{
                const finalValue = parseFloat(element.textContent);
                let currentValue = 0;
                const increment = finalValue / 50;
                const timer = setInterval(() => {{
                    currentValue += increment;
                    if (currentValue >= finalValue) {{
                        currentValue = finalValue;
                        clearInterval(timer);
                    }}
                    element.innerHTML = currentValue.toFixed(4) + '<br><small>' + 
                                      element.querySelector('small').textContent + '</small>';
                }}, 20);
            }});
        }});
    </script>
</body>
</html>
"""
        
        return html_content

    # === 统一对比函数 ===
    def generate_multi_strategy_comparison(self):
        try:
            pnl1, sharpe1 = self.strategy_fixed_threshold()
        except Exception as e:
            print(f"Fixed Threshold Strategy Error: {e}")
            pnl1, sharpe1 = pd.Series([0]*len(self.price), index=self.price.index), 0
            
        try:
            pnl2, sharpe2 = self.strategy_improved_threshold()
        except Exception as e:
            print(f"Improved Threshold Strategy Error: {e}")
            pnl2, sharpe2 = pd.Series([0]*len(self.price), index=self.price.index), 0
            
        try:
            pnl3, sharpe3 = self.strategy_adaptive_threshold()
        except Exception as e:
            print(f"Adaptive Threshold Strategy Error: {e}")
            pnl3, sharpe3 = pd.Series([0]*len(self.price), index=self.price.index), 0
            
        try:
            pnl4, sharpe4 = self.strategy_adaptive_trend()
        except Exception as e:
            print(f"Adaptive Threshold + Trend Strategy Error: {e}")
            pnl4, sharpe4 = pd.Series([0]*len(self.price), index=self.price.index), 0
            
        try:
            pnl5, sharpe5 = self.strategy_dynamic_position()
        except Exception as e:
            print(f"Dynamic Position Strategy Error: {e}")
            pnl5, sharpe5 = pd.Series([0]*len(self.price), index=self.price.index), 0
            
        try:
            pnl6, sharpe6 = self.strategy_volatility_weighted()
        except Exception as e:
            print(f"Volatility Weighted Strategy Error: {e}")
            pnl6, sharpe6 = pd.Series([0]*len(self.price), index=self.price.index), 0
            
        try:
            pnl7, sharpe7 = self.strategy_vol_weighted_dynamic_threshold()
        except Exception as e:
            print(f"Vol Weighted + Dynamic Threshold Strategy Error: {e}")
            pnl7, sharpe7 = pd.Series([0]*len(self.price), index=self.price.index), 0
            
        try:
            pnl8, sharpe8 = self.strategy_multi_timeframe()
        except Exception as e:
            print(f"Multi Timeframe Strategy Error: {e}")
            pnl8, sharpe8 = pd.Series([0]*len(self.price), index=self.price.index), 0
            
        try:
            pnl9, sharpe9 = self.strategy_ml_enhanced()
        except Exception as e:
            print(f"ML Enhanced Strategy Error: {e}")
            pnl9, sharpe9 = pd.Series([0]*len(self.price), index=self.price.index), 0
            
        try:
            pnl10, sharpe10 = self.strategy_risk_parity()
        except Exception as e:
            print(f"Risk Parity Strategy Error: {e}")
            pnl10, sharpe10 = pd.Series([0]*len(self.price), index=self.price.index), 0

        # 找到最高的夏普比率
        sharpe_ratios = [sharpe1, sharpe2, sharpe3, sharpe4, sharpe5, sharpe6, sharpe7, sharpe8, sharpe9, sharpe10]
        max_sharpe = max(sharpe_ratios) if any(s != 0 for s in sharpe_ratios) else 0

        # 绘制图形
        plt.figure(figsize=(16,10))
        plt.plot(pnl1, label=f"Fixed Threshold Sharpe: {sharpe1:.4f}")
        plt.plot(pnl2, label=f"Improved Threshold Sharpe: {sharpe2:.4f}")
        plt.plot(pnl3, label=f"Adaptive Threshold Sharpe: {sharpe3:.4f}")
        plt.plot(pnl4, label=f"Adaptive + Trend Sharpe: {sharpe4:.4f}")
        plt.plot(pnl5, label=f"Dynamic Position Sharpe: {sharpe5:.4f}")
        plt.plot(pnl6, label=f"Volatility Weighted Sharpe: {sharpe6:.4f}", linewidth=2, linestyle="--")
        plt.plot(pnl7, label=f"Vol Weighted + Dynamic Threshold Sharpe: {sharpe7:.4f}", linewidth=2, linestyle=":")
        plt.plot(pnl8, label=f"Multi Timeframe Sharpe: {sharpe8:.4f}", linewidth=2, linestyle="-.")
        plt.plot(pnl9, label=f"ML Enhanced Sharpe: {sharpe9:.4f}", linewidth=3, linestyle=(0, (3, 1, 1, 1)))
        plt.plot(pnl10, label=f"Risk Parity Sharpe: {sharpe10:.4f}", linewidth=3, linestyle=(0, (5, 10)))

        plt.title(f"Cumulative Returns Comparison of Ten Strategies (Max Sharpe: {max_sharpe:.4f})")
        plt.legend()
        plt.grid(True)
        plt.xlabel("Date")
        plt.ylabel("Cumulative Return")
        plt.show()

        # 返回结果
        results = {
            "Fixed Threshold": sharpe1,
            "Improved Threshold": sharpe2,
            "Adaptive Threshold": sharpe3,
            "Adaptive Threshold + Trend": sharpe4,
            "Dynamic Position": sharpe5,
            "Volatility Weighted": sharpe6,
            "Vol Weighted + Dynamic Threshold": sharpe7,
            "Multi Timeframe": sharpe8,
            "ML Enhanced": sharpe9,
            "Risk Parity": sharpe10
        }
        
        return results

# ===== 使用示例 =====
# 更完善的错误处理
try:
    # 检查文件是否存在
    filepath = "MinutesIdx.h5"
    if not os.path.exists(filepath):
        print(f"Warning: File {filepath} does not exist")
        print("Please make sure the file exists in the current directory, or modify the file path in the code")
        # 创建示例数据用于测试
        print("Creating sample data for demonstration...")
        dates = pd.date_range('2020-01-01', periods=1000, freq='D')
        sample_data = pd.DataFrame({
            'Open': np.random.randn(1000).cumsum() + 100,
            'High': np.random.randn(1000).cumsum() + 101,
            'Low': np.random.randn(1000).cumsum() + 99,
            'Close': np.random.randn(1000).cumsum() + 100,
            'Volume': np.random.randint(1000, 10000, 1000)
        }, index=dates)
        
        # 保存示例数据
        sample_data.to_hdf(filepath, key='IC', mode='w')
        print(f"Sample data file created: {filepath}")
    
    mydata = stock_info(filepath, "IC")
    results = mydata.generate_multi_strategy_comparison()
    print("Sharpe Ratios of All Strategies:")
    max_sharpe = max(results.values()) if any(v != 0 for v in results.values()) else 0
    print(f"Maximum Sharpe Ratio: {max_sharpe:.4f}")
    print("-" * 60)
    for strategy, sharpe in results.items():
        print(f"{strategy:<35} {sharpe:.4f}")
    
    # 生成HTML5报告
    html_report = mydata.generate_html_report(results)
    
    # 保存HTML文件
    html_filename = "strategy_analysis_report.html"
    with open(html_filename, 'w', encoding='utf-8') as f:
        f.write(html_report)
    
    print(f"\n📊 HTML5分析报告已生成: {html_filename}")
    print(f"📂 请在浏览器中打开文件查看详细报告")
    
    # 显示新策略的特点
    print("\n=== 新增策略特点 ===")
    print("🔸 多时间框架策略: 结合5/20/60分钟信号，提高信号可靠性")
    print("🔸 机器学习增强: 使用随机森林分类器预测价格方向")
    print("🔸 风险平价策略: 控制组合波动率，动态调整杠杆")
    print("🔸 HTML5可视化: 专业的交互式分析报告")
    
except FileNotFoundError as e:
    print(f"File Error: {e}")
    print("Please check if the file path is correct")
    
except ValueError as e:
    print(f"Data Value Error: {e}")
    print("Please check if the HDF file structure is correct")
    print("Possible reasons:")
    print("1. The data structure in the HDF file does not match expectations")
    print("2. The target column does not exist or the data type does not match")
    print("3. The data contains non-numeric elements")
    
except Exception as e:
    print(f"Unexpected Error: {e}")
    import traceback
    traceback.print_exc()